In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
cd '/content/gdrive/My Drive/VIP_Project'

/content/gdrive/My Drive/VIP_Project


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pylab as plt

In [ ]:
num_epoch = 100
batch_size = 1000

in_timesteps  = range(0,19)
out_timesteps = range(1,20)

# prepare data
data = np.load( "moving-shapes-2-tr-images.npy" )

# training set
x_tr = data[:5000,in_timesteps]
y_tr = data[:5000,out_timesteps]
tr_set = data[0:5000, :, :, :]

# validation set
x_te = data[5000:6000,in_timesteps]
y_te = data[5000:6000,out_timesteps]
te_set = data[5000:6000, :, :, :]

#print (np.shape(x_tr), np.shape(y_tr), np.shape(x_te), np.shape(y_te))
print (np.shape(x_te), np.shape(y_te))

(1000, 19, 64, 64, 3) (1000, 19, 64, 64, 3)


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Add, Conv2D, Conv3D, Concatenate, ConvLSTM2D, BatchNormalization, LeakyReLU, MaxPooling2D, UpSampling2D, TimeDistributed

def mainmodel():
  # Inputs
  dtype='float32'  
  contentInput = Input(shape=(None, 64, 64, 3), dtype=dtype)

  #Encoding Network
  x1 = ConvLSTM2D(128, (5,5), padding='same', return_sequences=True, kernel_initializer ='he_normal')(contentInput)
  x2 = ConvLSTM2D(64, (5,5), padding='same', return_sequences=True, kernel_initializer ='he_normal')(x1) 
  x3 = ConvLSTM2D(64, (5,5), padding='same', return_sequences=True, kernel_initializer ='he_normal')(x2) 

  #Forecasting Network
  x4 = ConvLSTM2D(128, (5,5), padding='same', return_sequences=True, kernel_initializer ='he_normal')(x1)
  x5 = ConvLSTM2D(64, (5,5), padding='same', return_sequences=True, kernel_initializer ='he_normal')(x2)
  x6 = ConvLSTM2D(64, (5,5), padding='same', return_sequences=True, kernel_initializer ='he_normal')(x3)

  #Prediction Network
  conc = Concatenate()([x4, x5, x6]) #
  predictions = Conv3D(3, (5,5,5), activation='sigmoid', padding='same')(conc) #sigmoid original

  model = Model(inputs=contentInput, outputs=predictions)
  
  model.compile(optimizer='adam', loss='binary_crossentropy')
  return model

In [ ]:
seq = mainmodel()

In [ ]:

#seq.load_weights('conv_lstm_en_de_shapes') #Epoch 1

In [ ]:
seq.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 64, 64 0                                            
__________________________________________________________________________________________________
conv_lst_m2d (ConvLSTM2D)       (None, None, 64, 64, 1677312     input_1[0][0]                    
__________________________________________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)     (None, None, 64, 64, 1229056     conv_lst_m2d[0][0]               
__________________________________________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)     (None, None, 64, 64, 819456      conv_lst_m2d_1[0][0]             
_______________________________________________________________________________________

In [ ]:
def generator(X, y, batch_size):
    while True:
        for start in range(0, len(X), batch_size):
            end = min(start + batch_size, len(X))
            yield np.array(X[start:end]/255), np.array(y[start:end]/255)

In [ ]:
checkpoint_filepath = '/content/gdrive/My Drive/VIP_Project/conv_lstm_en_de_shapes_checkpoint/conv_lstm_en_de_shapes_checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode = min,
    save_best_only=True,
    verbose=1
  )

In [ ]:
batch_size = 4

epo = 1
history =seq.fit_generator(
    generator(x_tr, y_tr, batch_size),
    steps_per_epoch = len(x_tr)/batch_size,
    epochs= epo,
    verbose=1,
    validation_data = generator(x_te, y_te, batch_size),
    validation_steps = len(x_te)/batch_size,
    callbacks=[model_checkpoint_callback]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5


ResourceExhaustedError: ignored

In [ ]:
for sequence in range(1,7,1):
    track = x_te[sequence, :20, ::, ::, ::]
    y_prediction = seq.predict(track[np.newaxis])
    #print(y_prediction.shape)
    # ground truth
    plt.figure(figsize=(20,10))
    for i in range(20):
        plt.subplot(1, 20, i+1)
        plt.tick_params(
        axis='both',       # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off', # labels along the bottom edge are off
        labelleft='off')
        plt.imshow(te_set[sequence,i,:,:,:])

    # prediction
    plt.figure(figsize=(20,10))
    plt.subplot(1,20,1)
    plt.imshow(np.zeros((64,64,3)))
    for i in range(19):
        plt.subplot(1, 20, i+2) 
        plt.tick_params(
        axis='both',       # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off', # labels along the bottom edge are off
        labelleft='off')
        plt.imshow(y_prediction[0,i,:,:,:])

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from skimage.metrics import structural_similarity as ssim
from skimage import img_as_float32

te_set = img_as_float32(te_set)
y_prediction = seq.predict(x_te[:, :20, ::, ::, ::])

mse=0
mae=0
ssim_v=0

for i in range(batch_size):
  mse_i=0
  mae_i=0
  ssim_i=0
  for j in range(19):
    mse_total=0
    mae_total=0
    ssim_total=0
    for k in range(3):
      mse_total += mean_squared_error(te_set[i,j+1,:,:,k],y_prediction[i,j,:,:,k])
      mae_total += mean_absolute_error(te_set[i,j+1,:,:,k],y_prediction[i,j,:,:,k])
      ssim_total += ssim(te_set[i,j+1,:,:,k],y_prediction[i,j,:,:,k],data_range=y_prediction[i,j,:,:,k].max() - y_prediction[i,j,:,:,k].min())
    mse_i += mse_total/3
    mae_i += mae_total/3
    ssim_i += ssim_total/3

  mse += mse_i
  mae += mae_i
  ssim_v += ssim_i

mse = mse/(batch_size*19)
mae = mae/(batch_size*19)
ssim_v = ssim_v/(batch_size*19)

print('MSE: ',mse)
print('MAE: ',mae)
print('SSIM: ',ssim_v)